In [ ]:
import time

def TicTocGenerator():
    # Generator that returns time differences
    ti = 0           # initial time
    tf = time.time() # final time
    while True:
        ti = tf
        tf = time.time()
        yield tf-ti # returns the time difference

TicToc = TicTocGenerator() # create an instance of the TicTocGen generator

# This will be the main function through which we define both tic() and toc()
def toc(tempBool=True):
    # Prints the time difference yielded by generator instance TicToc
    tempTimeInterval = next(TicToc)
    if tempBool:
        print( "Elapsed time: %f seconds.\n" %tempTimeInterval )

def tic():
    # Records a time in TicToc, marks the beginning of a time interval
    toc(False)

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from torch import nn, optim
from math import exp

In [ ]:
# initialize parameters
# dx is the step size in test set

global k, dx, beta 
k, dx, beta = 10, .001, 1

In [ ]:
def f(x):
    if x< 0.5 :
        return 8*k*(3*x-1)
    else :
        return 4*k*(k+1)
    
def g(x):
    return torch.tensor([0.], requires_grad=True)

def u_exact(x):
    if x< 0.5 :
        return 4*k*x**2*(1-x)
    else :
        return (2*(k+1)*x-1)*(1-x)

def sigma_exact(x):
    if x< 0.5 :
        return -12*k*x**2+8*k*x
    else :
        return -4*(k+1)*x + 2 *k + 3

sq = lambda x: x ** 2
vsq = np.vectorize(sq)

In [ ]:
# compute H1 norm of true u and sigma
L = 0.
R = 1.
test_set1 =  np.arange(L, R/2, dx)
test_set2 =  np.arange(R/2, R, dx)
test_set = np.concatenate((test_set1, test_set2))
u1 = np.vectorize(u_exact)(test_set1)
ud1 = np.vectorize(sigma_exact)(test_set1)
u2 = np.vectorize(u_exact)(test_set2)
ud2 = np.vectorize(sigma_exact)(test_set2)
u_h1 = np.sum(dx*(vsq(ud1)+ vsq(ud2) ))
u_l2 = np.sum(dx*vsq(u1)+ dx*vsq(u2) )

sigma1 = np.vectorize(sigma_exact)(test_set1)
sigmad1 = -np.vectorize(f)(test_set1)
sigma2 = np.vectorize(sigma_exact)(test_set2)
sigmad2 = -np.vectorize(f)(test_set2)/k

sigma_h1 = np.sum(dx*(vsq(sigma1) + vsq(sigmad1) + vsq(sigma2) + vsq(sigmad2)))
sigma_l2 = np.sum(dx*vsq(sigma1) + dx*vsq(sigma2) )

print('u: H1 norm square: %.6f, L2 norm square: %.6f ' %(u_h1, u_l2))
print('sigma: H1 norm square: %.6f, L2 norm square: %.6f ' %(sigma_h1, sigma_l2))

In [ ]:
class MuSigmaPde(nn.Module):
    def __init__(self, dimension, mesh = 32, neuron = 24):
        super(MuSigmaPde, self).__init__()

        self.xdim = dimension
        # Layer 1
        self.fc1mu = nn.Linear(dimension, mesh)
        #self.fc1sig = nn.Linear(dimension, mesh)
        # Layer 2
        self.fc2mu = nn.Linear(mesh, neuron)
        #self.fc2sig = nn.Linear(mesh, neuron)
        # Layer 3
        self.fc3mu = nn.Linear(neuron, neuron)
        #self.fc3sig = nn.Linear(neuron, neuron)
        # Layer 4
        self.fc4mu = nn.Linear(neuron, 1)
        #self.fc4sig = nn.Linear(neuron, dimension)

    def forward(self, x):   #Activation function sigmoid
        assert(len(x.shape) == 1 and x.shape[0] == self.xdim)
        mu =  torch.sigmoid(self.fc2mu(torch.sigmoid(self.fc1mu(x))))
        mu =  self.fc4mu(torch.sigmoid(self.fc3mu(mu)))
        return mu
    
    
    def net_grad(self, x, h):
        mu_center = self.forward(x)
        mu_forward = self.forward(x - .5*h)
        mu_backward =self.forward(x + 0.5*h)

        mu_grad_forward = (mu_center - mu_forward)/(.5*h)
        mu_grad_backward = (mu_backward - mu_center)/(0.5*h)
        mu_lap = (mu_grad_backward - mu_grad_forward)/ (0.5*h)       
    
        return mu_grad_forward, mu_lap     
    
    def loss_function_bulk_ls(self, x,h):   #LS functional
        mu_grad_forward, mu_lap = self.net_grad(x,h)
        if x< 0.5:
            LSE = (mu_lap + f(x))**2
        else: 
            LSE = (mu_lap *k + f(x))**2
        return LSE 
    
    def loss_function_bulk_en(self, x,h):   #Energy functional
        mu = self.forward(x)
        mu_grad_forward, mu_lap = self.net_grad(x,h)
        if x< 0.5:
            LSE = 0.5 * mu_grad_forward**2 - f(x)*mu
        else: 
            LSE = 0.5 * k * mu_grad_forward**2 - f(x)*mu
        return LSE 

    def loss_function_surf(self, x):
        mu = self.forward(x)
        # Boundary condition penalty
        BCP = beta * (mu - g(x))**2
        return BCP

In [ ]:
model = MuSigmaPde(dimension =1, mesh = 32, neuron = 24)

In [ ]:
sum([p.numel() for p in model.parameters()])

In [ ]:
h= .002
L, R = 0., 1.
epochs = 20000
bulk_set, surf_set =  np.arange(L, R, h), [L, R]
loss_bulk_record, loss_surf_record = [], []
print('bulk points number %d \nsurface points number %d\ntest points number %d\ndx for difference in testing %.3g\ntrainging iteration %d' %(np.size(bulk_set), np.size(surf_set), np.size(test_set), dx, epochs))

In [ ]:
def exp_lr_scheduler(optimizer, epoch, lr_decay=0.1, lr_decay_epoch=10000):
    """Decay learning rate by a factor of lr_decay every lr_decay_epoch epochs"""
    if epoch % lr_decay_epoch:
        return optimizer
    if epoch == 0:
        return optimizer
    
    for param_group in optimizer.param_groups:
        param_group['lr'] *= lr_decay
    return optimizer

optimizer = optim.Adam(model.parameters(), lr = 0.01)

In [ ]:
tic()
local_min = -135
for j in range(epochs):
    loss_bulk = torch.zeros(1)
    loss_surf = torch.zeros(1)

    for point in bulk_set:
        x = torch.tensor([point+ 0.5*h])
        loss_bulk += h*model.loss_function_bulk_ls(x,h)
               
    for point in surf_set:
        x = torch.tensor([point])
        loss_surf += model.loss_function_surf(x)

    loss_bulk_record.append(loss_bulk.data[0])
    loss_surf_record.append(loss_surf.data[0])
       
    loss = loss_bulk + loss_surf        
    print('Train Epoch: {}, Loss: {:.6f}, loss bulk: {:.6f}, loss surf: {:.6f}'.format(j, loss.item(), loss_bulk.item(), loss_surf.item()))
    optimizer.zero_grad()
    loss.backward()
    exp_lr_scheduler(optimizer, j)
    
    if loss.item() < local_min:
        print('updating the parameters')
        local_min = loss.item()
        torch.save(model.state_dict(),'./diffusion_sigmoid_en')

    optimizer.step()   
toc() 


In [ ]:
model.load_state_dict(torch.load('./diffusion_sigmoid_en'))

In [ ]:
mu_err_h1 = torch.zeros(1)
sigma_err_h1 = torch.zeros(1)
bdd_err = torch.zeros(1)
mu_err_l2 = torch.zeros(1)
sigma_err_l2 = torch.zeros(1)
G_relative = torch.zeros(1)
mu_err_semi = torch.zeros(1)

for point in test_set:
    x = torch.tensor([point+ 0.5*dx])
    mu = model(x)
    mu_grad, mu_lap = model.net_grad(x,dx)

    # esitmate H1 norm error
    mu_diff_simi = (mu_grad - sigma_exact(x))**2
    
    # estimate L2 norm error
    mu_err_l2 += dx*(mu - u_exact(x))**2
    
    # estimate H1 semi norm  error
    mu_err_semi += dx*mu_diff_simi
    


mu_err_l2_relative = (mu_err_l2/u_l2)**(1/2)
mu_err_semi_relative = (mu_err_semi/(sigma_l2))**(1/2)

print('u: L2_rel: {:.6f}, H1_semi_rel: {:.6f}'.format( mu_err_l2_relative.item(), mu_err_semi_relative.item()))

In [ ]:
points = test_set
yt = np.zeros_like(points)
y_diff = np.zeros_like(points)
ymu = np.zeros_like(points)
ysig = np.zeros_like(points)
for i in range(len(points)):
    yt[i] = u_exact(points[i])
    y_diff[i] =  sigma_exact(points[i])
    ymu[i] = model(torch.tensor([points[i]]))
    ysig[i], ss = model.net_grad(torch.tensor([points[i]]),dx)

In [ ]:
plt.plot(points, yt, color = 'b', label = 'u_true')
plt.plot(points, ymu, color = 'r', label = 'u_approximation')
plt.ylim([0,7])
plt.legend()

In [ ]:
plt.plot(points, y_diff, color = 'b', label = 'u\'_true')
plt.plot(points, ysig, color = 'r', label = 'u\'_approximation')

In [ ]:
num = np.arange(1, len(loss_bulk_record)+1, 1)
plt.plot(num, loss_bulk_record)
plt.plot(num, loss_surf_record)
plt.plot(num, np.add(loss_bulk_record , loss_surf_record))